In [8]:
import hashlib 
def compute_hash(path, algorithm): 
    h = hashlib.new(algorithm) 
    with open(path, "rb") as f: 
        h.update(f.read()) 
    return h.hexdigest() 
sample = r"C:\Users\blabb\Portfolio\Procmon.exe" 
print("MD5:   ", compute_hash(sample, "md5")) 
print("SHA1:  ", compute_hash(sample, "sha1")) 
print("SHA256:", compute_hash(sample, "sha256"))

MD5:    3b493c33bc35f2b0ecf3c354d58131d6
SHA1:   f92ab0bebde636ece6f7bb191d3b2b12c682c085
SHA256: 883b2f76712d9c3edd2bfd51443d37b62ab66c5b637d161753465ff912a86043


In [3]:
import re 
def extract_strings(path): 
    with open(path, "rb") as f: 
        data = f.read() 
    pattern = rb"[ -~]{4,}" 
    return re.findall(pattern, data) 
strings = extract_strings(sample) 
for s in strings[:20]: 
    print(s.decode(errors="ignore")) 

!This program cannot be run in DOS mode.
V*0T
0RichU
.text
`.rdata
@.data
.rsrc
@.reloc
hpqQ
h`EN
h|nN
h\nN
hlnN
=UUU
h_rM
hDLN
h`GO
hDLN
h|GO
hDLN


In [5]:
import pefile 
pe = pefile.PE(sample) 
print("Entry Point:", hex(pe.OPTIONAL_HEADER.AddressOfEntryPoint)) 
print("Image Base:", hex(pe.OPTIONAL_HEADER.ImageBase)) 
print("\nImported DLLs and functions:") 
for entry in pe.DIRECTORY_ENTRY_IMPORT: 
    print(" ", entry.dll.decode()) 
    for imp in entry.imports[:5]: 
        print("     -", imp.name.decode() if imp.name else "None") 

Entry Point: 0xa7f70
Image Base: 0x400000

Imported DLLs and functions:
  WS2_32.dll
     - getsockname
     - listen
     - recv
     - closesocket
     - socket
  VERSION.dll
     - GetFileVersionInfoW
     - VerQueryValueW
     - GetFileVersionInfoSizeW
  COMCTL32.dll
     - ImageList_ReplaceIcon
     - ImageList_SetBkColor
     - ImageList_AddMasked
     - ImageList_BeginDrag
     - ImageList_EndDrag
  FLTLIB.DLL
     - FilterSendMessage
     - FilterGetMessage
     - FilterReplyMessage
     - FilterConnectCommunicationPort
  KERNEL32.dll
     - AcquireSRWLockExclusive
     - AcquireSRWLockShared
     - InitializeSRWLock
     - GetSystemInfo
     - VerSetConditionMask
  USER32.dll
     - LoadStringA
     - DrawEdge
     - GetMessageW
     - TranslateMessage
     - DispatchMessageW
  GDI32.dll
     - SaveDC
     - RestoreDC
     - SetBrushOrgEx
     - SetPixel
     - PatBlt
  COMDLG32.dll
     - ChooseColorW
     - GetOpenFileNameW
     - PrintDlgW
     - ChooseFontW
     - FindText

In [9]:
import yara 
rule_source = """ 
rule ContainsHTTP { 
strings: 
$s = "http" 
condition: 
$s 
} 
""" 
rules = yara.compile(source=rule_source) 
matches = rules.match(sample) 
print(matches) 

Failed to import 'c:\Users\blabb\Portfolio\.venv\DLLs\libyara.dll'
PATH = c:\Users\blabb\Portfolio\.venv\Scripts;C:\Program Files (x86)\Common Files\Oracle\Java\java8path;C:\Program Files (x86)\Common Files\Oracle\Java\javapath;C:\Windows\system32;C:\Windows;C:\Windows\System32\Wbem;C:\Windows\System32\WindowsPowerShell\v1.0\;C:\Windows\System32\OpenSSH\;C:\Program Files (x86)\NVIDIA Corporation\PhysX\Common;C:\Program Files\NVIDIA Corporation\NVIDIA app\NvDLISR;C:\Program Files\Git\cmd;C:\Program Files\nodejs\;C:\Program Files\dotnet\;C:\Users\blabb\AppData\Local\Microsoft\WindowsApps;C:\Users\blabb\AppData\Local\Programs\Microsoft VS Code\bin;C:\Users\blabb\AppData\Roaming\npm;C:\Users\blabb\AppData\Local\Programs\mongosh\;C:\Users\blabb\AppData\Local\Programs\MiKTeX\miktex\bin\x64\;C:\Users\blabb\AppData\Local\Python\bin;C:\Users\blabb\AppData\Local\Programs\Ollama;c:\Users\blabb\Portfolio\.venv\DLLs;c:\Users\blabb\Portfolio\.venv\DLLs;c:\Users\blabb\Portfolio\.venv\DLLs;c:\Users\bl

PermissionError: [WinError 5] Access is denied

In [1]:
import hashlib, pefile, re, yara 
# sample = "samples/procmon.exe" 
def compute_hashes(path): 
    algos = ["md5", "sha1", "sha256"] 
    output = {} 
    for a in algos: 
        h = hashlib.new(a) 
        with open(path, "rb") as f: 
            h.update(f.read()) 
        output[a] = h.hexdigest() 
    return output 
def extract_strings(path): 
    with open(path, "rb") as f: 
        data = f.read() 
    return re.findall(rb"[ -~]{4,}", data) 
print("Hashes:", compute_hashes(sample)) 
print("\nStrings:") 
print(extract_strings(sample)[:10]) 
print("\nImports:") 
pe = pefile.PE(sample) 
for entry in pe.DIRECTORY_ENTRY_IMPORT: 
    print(entry.dll.decode()) 
print("\nIOCs:") 
decoded = open(sample, "rb").read().decode(errors="ignore") 
print("URLs:", re.findall(r"https?://[^\s\"']+", decoded)) 
print("IPs:", re.findall(r"\b\d{1,3}(?:\.\d{1,3}){3}\b", decoded)) 
print("\nYARA:") 
rule = yara.compile(source=""" 
rule Simple { 
strings: $s = "http" 
condition: $s 
} 
""") 
print(rule.match(sample))

Failed to import 'c:\Users\blabb\Portfolio\.venv\DLLs\libyara.dll'
PATH = c:\Users\blabb\Portfolio\.venv\Scripts;C:\Program Files (x86)\Common Files\Oracle\Java\java8path;C:\Program Files (x86)\Common Files\Oracle\Java\javapath;C:\Windows\system32;C:\Windows;C:\Windows\System32\Wbem;C:\Windows\System32\WindowsPowerShell\v1.0\;C:\Windows\System32\OpenSSH\;C:\Program Files (x86)\NVIDIA Corporation\PhysX\Common;C:\Program Files\NVIDIA Corporation\NVIDIA app\NvDLISR;C:\Program Files\Git\cmd;C:\Program Files\nodejs\;C:\Program Files\dotnet\;C:\Users\blabb\AppData\Local\Microsoft\WindowsApps;C:\Users\blabb\AppData\Local\Programs\Microsoft VS Code\bin;C:\Users\blabb\AppData\Roaming\npm;C:\Users\blabb\AppData\Local\Programs\mongosh\;C:\Users\blabb\AppData\Local\Programs\MiKTeX\miktex\bin\x64\;C:\Users\blabb\AppData\Local\Python\bin;C:\Users\blabb\AppData\Local\Programs\Ollama;c:\Users\blabb\Portfolio\.venv\DLLs


PermissionError: [WinError 5] Access is denied